In [ ]:
# Code to mount the drive 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#For Himanshu's access
import os
# path = "/content/drive/MyDrive/685/catr"
# os.chdir(path)
path = "/content/drive/MyDrive/CollabProjects/685/catr"
os.chdir(path)
print(os.getcwd())
!nvidia-smi

/content/drive/MyDrive/CollabProjects/685/catr
Thu Dec 16 21:57:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                         

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 35.1 MB/s 
     |████████████████████████████████| 596 kB 65.5 MB/s 
     |████████████████████████████████| 61 kB 683 kB/s 
     |████████████████████████████████| 3.3 MB 47.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from transformers import BertTokenizer,BertForMaskedLM
from PIL import Image
import argparse
from models import caption
from datasets import coco, utils
from configuration import Config
import os

In [51]:
##Initializing variables

prediction_type = "sarcasm" # Options: ["sarcasm", "emogen"]
checkpoint_name = "BaselineEmogenTestSetResults" # Options: ["checkpoint_base_emogen.pth", "checkpoint_base_sarcasm.pth", "checkpoint_emogen2sarcasm", "checkpoint_sarcasm2emogen", "checkpoint_sarcasm_back_trans.pth", "checkpoint_sarcasm_back_trans_2.pth"]
checkpoint_path = checkpoint_basepath + checkpoint_name + ".pth"

if prediction_type == "sarcasm":
  print("Chosen type:", prediction_type)
  image_path = '../sarcasm/test/'
  images_mappings = r'../sarcasm/annotations/captions_test.json'
  reference_data_file_path = '../sarcasm/test_data/caption/test_caption.csv'

elif prediction_type == "emogen":
  print("Chosen type:", prediction_type)
  image_path = '../emogen/test/'
  images_mappings = r'../emogen/annotations/caption_test.json'
  reference_data_file_path = '../emogen/annotations/test_caption.csv'

Chosen type: sarcasm


In [ ]:
# Create model configurations 
config = Config()
# Check if the model is able to find the checkpoint
if not os.path.exists(checkpoint_path):
  raise NotImplementedError('Give valid checkpoint path')

In [ ]:
# Load the finetuned model in our case
model,_ = caption.build_model(config)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [ ]:
# Map location to CPu to unserialize the model using CPU 
checkpoint = torch.load(checkpoint_path, map_location='cpu')
# Load model dictionary 
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [ ]:
# Load BERT tokenizer
tokenizer   = BertTokenizer.from_pretrained('bert-base-uncased')
start_token = tokenizer.convert_tokens_to_ids(tokenizer._cls_token)
end_token = tokenizer.convert_tokens_to_ids(tokenizer._sep_token)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def create_caption_and_mask(start_token, max_length):
    caption_template = torch.zeros((1, max_length), dtype=torch.long)
    mask_template = torch.ones((1, max_length), dtype=torch.bool)

    caption_template[:, 0] = start_token
    mask_template[:, 0] = False

    return caption_template, mask_template

In [ ]:
@torch.no_grad()
def evaluate(image, caption, cap_mask):
    model.eval()
    for i in range(config.max_position_embeddings - 1):
        predictions = model(image, caption, cap_mask)
        predictions = predictions[:, i, :]
        predicted_id = torch.argmax(predictions, axis=-1)

        if predicted_id[0] == 102:
            return caption

        caption[:, i+1] = predicted_id[0]
        cap_mask[:, i+1] = False

    return caption

In [ ]:
import json

f = open(images_mappings)
image_details = json.load(f)["annotations"]

result_json = {"predicted_captions":[]}

#index = 0
for image_detail in image_details:
  image_name = image_detail['image_id']+'.jpg'
  img_final_loc = image_path+image_name
  # Load image and perform pre-processing 

  caption, cap_mask = create_caption_and_mask(
    start_token, config.max_position_embeddings)

  image = Image.open(img_final_loc)
  image = coco.val_transform(image)
  image = image.unsqueeze(0)

  output = evaluate(image, caption, cap_mask)
  result = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)

  result_json["predicted_captions"].append({"image": image_detail['image_id'], "caption": result})

  #index+=1

  #if index>100:
   # break

/content/drive/MyDrive/CollabProjects/685/catr/models/position_encoding.py:38: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


In [ ]:
# GPT-2 for calculation of the requisite perplexity 
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
device = 'cpu'
model_id = 'gpt2-large'
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
model.eval()

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [53]:
##loading predicted data
import json
import csv
predicted_image_mappings =  r'./IntermediateFineTuning/'+checkpoint_name+'.json'
f = open(predicted_image_mappings)
image_details = json.load(f)["annotations"]

In [54]:
image_details

[{'caption': "July Com'on mixing mixing mixing cooked move woahh equipment. baby!! baby!! GO, kudos several several several several haunted in We team skin skin handsome flowers! clothes! flowers! flowers! flowers! cows! dishes branch branch branch $120,000 WOMEN'S WOMEN'S Impossible! Best Best soak in smile public, collect visiting visiting visiting how gorgeous room! room! We talented ",
  'image': 'be877e8b958869c6751578f137139d7c'},
 {'caption': "July Com'on mixing mixing mixing cooked move woahh equipment. baby!! baby!! GO, kudos several several several several haunted in We team skin skin handsome flowers! clothes! flowers! flowers! flowers! cows! dishes branch branch branch $120,000 WOMEN'S WOMEN'S Impossible! Best Best soak in smile public, collect visiting visiting visiting how gorgeous room! room! We talented ",
  'image': '43998d5eee707c76ae879c3bf733764b'},
 {'caption': "July Com'on mixing mixing mixing cooked move woahh equipment. baby!! baby!! GO, kudos several several se

In [ ]:
##Calulating average perplexity of generated captions

from tqdm import tqdm
import torch
result_json = {"predicted_captions": []}

for image_detail in image_details:
  if len(image_detail["caption"].split(" "))>=6:
    result_json['predicted_captions'].append(image_detail)

print(result_json)


total_ppl = 0
total_count = 0
perplexities = []
for image_map in result_json["predicted_captions"]:
  encodings = tokenizer(''.join(image_map["caption"]), return_tensors='pt')
  max_length = model.config.n_positions
  stride = 3

  lls = []
  for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
      # print(encodings.input_ids.size(1))
      begin_loc = max(i + stride - max_length, 0)
      end_loc = min(i + stride, encodings.input_ids.size(1))
      trg_len = end_loc - i    # may be different from stride on last loop
      input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
      target_ids = input_ids.clone()
      target_ids[:,:-trg_len] = -100

      with torch.no_grad():
          outputs = model(input_ids, labels=target_ids)
          log_likelihood = outputs[0] * trg_len

      lls.append(log_likelihood)

  ppl = torch.exp(torch.stack(lls).sum() / end_loc)
  perplexities.append(ppl)
  print(image_map["caption"], ppl)
  total_ppl+=ppl
  total_count+=1
  if total_count>20:
    break
print("The average perplexity is %s" % (total_ppl/total_count))

##Saving the perplexity in the result json
result_json["perplexity"] = float(total_ppl/total_count)

{'predicted_captions': [{'caption': "July Com'on mixing mixing mixing cooked move woahh equipment. baby!! baby!! GO, kudos several several several several haunted in We team skin skin handsome flowers! clothes! flowers! flowers! flowers! cows! dishes branch branch branch $120,000 WOMEN'S WOMEN'S Impossible! Best Best soak in smile public, collect visiting visiting visiting how gorgeous room! room! We talented ", 'image': 'be877e8b958869c6751578f137139d7c'}, {'caption': "July Com'on mixing mixing mixing cooked move woahh equipment. baby!! baby!! GO, kudos several several several several haunted in We team skin skin handsome flowers! clothes! flowers! flowers! flowers! cows! dishes branch branch branch $120,000 WOMEN'S WOMEN'S Impossible! Best Best soak in smile public, collect visiting visiting visiting how gorgeous room! room! We talented ", 'image': '43998d5eee707c76ae879c3bf733764b'}, {'caption': "July Com'on mixing mixing mixing cooked move woahh equipment. baby!! baby!! GO, kudos s

100%|██████████| 28/28 [00:41<00:00,  1.47s/it]


July Com'on mixing mixing mixing cooked move woahh equipment. baby!! baby!! GO, kudos several several several several haunted in We team skin skin handsome flowers! clothes! flowers! flowers! flowers! cows! dishes branch branch branch $120,000 WOMEN'S WOMEN'S Impossible! Best Best soak in smile public, collect visiting visiting visiting how gorgeous room! room! We talented  tensor(764.7174)


100%|██████████| 28/28 [00:40<00:00,  1.45s/it]


July Com'on mixing mixing mixing cooked move woahh equipment. baby!! baby!! GO, kudos several several several several haunted in We team skin skin handsome flowers! clothes! flowers! flowers! flowers! cows! dishes branch branch branch $120,000 WOMEN'S WOMEN'S Impossible! Best Best soak in smile public, collect visiting visiting visiting how gorgeous room! room! We talented  tensor(764.7174)


...

100%|██████████| 28/28 [00:40<00:00,  1.46s/it]


Giant Wrap Wrap Giant Wrap entranced Dang joy, joy, joy, star. other's other's other's moment other's other's shining Who Who exciting! cares bus! minion minion hit. hit. hit. rascals! thru thru thru pleasing house, house, trumpet traffic, hyper hyper kayak kayak sat Can Can Can alone! alone! Fairies TV? hills.  tensor(345.9352)


100%|██████████| 53/53 [02:15<00:00,  2.55s/it]


oooooo. Fresh Fresh keys Want I’m alley character. character. character. Their Simply Simply Simply lasts. romantic! romantic! speak! parking parking restoring! Sundance fixer-upper cathedrals cathedrals center center center lane hats, hats, hats, heaven shiny shiny 3...2... cleaned Their Their WHATEVER WHATEVER greenery! OTHER OTHER OTHER runners runners Are farm  family. family. family. freshest posing yummy Take wildly whats wildly Vacation Vacation activity. schools family. family.  or or or or time. animals! really. symphonic youthful symphonic youthful parking restoring! restoring! capitals stained stained PLACE of! they'd castle. castle. closer, bun workers Welcome Welcome contained Pan complicated; lanes  tensor(551.8916)


 74%|███████▎  | 39/53 [01:16<00:48,  3.45s/it]

In [ ]:
perplexities.sort()
perplexities

[tensor(14.0332),
 tensor(14.0332),
 tensor(14.4134),
 tensor(16.0138),
 tensor(17.4132),
 tensor(17.4132),
 tensor(17.5116),
 tensor(25.5323),
 tensor(27.9925),
 tensor(29.0616),
 tensor(30.0464),
 tensor(32.6241),
 tensor(35.6850),
 tensor(38.7311),
 tensor(40.1201),
 tensor(44.1679),
 tensor(47.5687),
 tensor(53.3235),
 tensor(53.6019),
 tensor(63.5542),
 tensor(63.9709),
 tensor(65.6855),
 tensor(69.4204),
 tensor(72.7473),
 tensor(72.9859),
 tensor(73.7705),
 tensor(74.7031),
 tensor(76.8410),
 tensor(78.8381),
 tensor(83.3942),
 tensor(83.9676),
 tensor(86.7213),
 tensor(88.2262),
 tensor(90.6361),
 tensor(92.5804),
 tensor(92.6430),
 tensor(93.6825),
 tensor(93.9598),
 tensor(97.0219),
 tensor(98.9787),
 tensor(107.9710),
 tensor(113.4414),
 tensor(115.3987),
 tensor(117.7418),
 tensor(119.4861),
 tensor(122.6686),
 tensor(122.9978),
 tensor(124.4176),
 tensor(135.3158),
 tensor(136.6632),
 tensor(141.7626),
 tensor(153.3487),
 tensor(161.9041),
 tensor(178.4262),
 tensor(183.25

In [ ]:
# ##Creating image and caption map for the result
# import csv
# from ast import literal_eval

# predicted_image_caption_map = {}
# for caption_result in result_json["predicted_captions"]:
#   caption = caption_result["caption"]
#   image = caption_result["image"]

#   predicted_image_caption_map[image] = caption

# ##Creating image and caption map for the reference test data

# def make_reference_map(csvFilePath):
#     # create a dictionary
#     reference_map = {}

#     # Open a csv reader called DictReader
#     with open(csvFilePath, encoding='utf-8') as csvf:
#         csvReader = csv.DictReader(csvf)
        
#         for rows in csvReader:

#             caption = literal_eval(rows["candidates"])
#             image_id = rows["image_hash"]
#             # print("a")
#             reference_map[image_id] = caption

#     return reference_map
    
# reference_image_caption_map = make_reference_map(reference_data_file_path)

In [ ]:
##Calculating the Blue Score
# from nltk.translate.bleu_score import sentence_bleu

# score = 0
# count = 0
# for image in predicted_image_caption_map:
#   predicted_caption = predicted_image_caption_map[image]
#   reference_captions = reference_image_caption_map[image]
#   blue_score = sentence_bleu(reference_captions, predicted_caption )
#   print('BLEU score -> {}'.format(blue_score))
#   score+=blue_score
#   count+=1

# average_blue_score = score/count
# print('Average BLEU score -> {}'.format(average_blue_score))
# result_json["blue_score"] = average_blue_score

In [ ]:
##Storing the results
path = checkpoint_basepath + checkpoint_name + ".json"
with open(path, 'w') as outfile:
    json.dump(result_json, outfile)